In [5]:
import os
os.environ["NAPARI_ASYNC"] = "1"

In [ ]:
!ls /Users/fukai

In [6]:
import zarr
from dask import array as da
image = da.from_zarr("/Users/fukai/image.zarr")
labels = da.from_zarr("/Users/fukai/labels.zarr")

In [3]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
import numpy as np
import tensorstore as ts
import trackarray_tensorstore as tats
from pathlib import Path

basedir = "/Users/fukai/"

spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": basedir+"labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = basedir + "image.zarr"
image = ts.open(image_spec).result()[:,:,:]

print(labels.shape)
bboxes_df_path = Path(basedir+"bbox.csv")
properties_path = Path(basedir+"splits.json")
ta = tats.read_files(labels, bboxes_df_path, properties_path)

(10, 234, 234)


In [ ]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
widget = StateMachineWidget(viewer, ta, image, 
                            verified_track_ids=ta.attrs.get("verified_track_ids", []),
                            candidate_track_ids=ta.attrs.get("candidate_track_ids", []),
                            crop_size=100)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [  0 143 180]
INFO:napari_travali2._logging:Region selected: coords [  0 143 180]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(93), np.int64(193), None), slice(np.int64(130), np.int64(230), None))
INFO:napari_travali2._logging:()
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(3.0), np.float64(144.66757555054843), np.float64(180.98630938502362))
DEBUG:napari_travali2._logging:data coordinates: [ 3.         51.66757555 50.98630939]
INFO:napari_travali2._logging:clicked at [ 3 52 51] at frame 3 and label value 2
INFO:napari_travali2._logging:Track selected: frame 3 value 2
INFO:napari_travali2._logging:()
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(5.0)